In [2]:
%load_ext autoreload

In [7]:
%autoreload
import sys

sys.path.insert(0, '../')
from constants import *
from utilities_RF import *
from utilities import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import GridSearchCV


In [8]:
selected_features = ['date_forecast',
 'absolute_humidity_2m:gm3',
 'air_density_2m:kgm3',
 'clear_sky_energy_1h:J',
 'clear_sky_rad:W',
 'dew_point_2m:K',
 'diffuse_rad:W',
 'diffuse_rad_1h:J',
 'direct_rad:W',
 'direct_rad_1h:J',
 'effective_cloud_cover:p',
 'is_day:idx',
 'is_in_shadow:idx',
 'prob_rime:p',
 'relative_humidity_1000hPa:p',
 'sfc_pressure:hPa',
 'sun_azimuth:d',
 'sun_elevation:d',
 'super_cooled_liquid_water:kgm2',
 't_1000hPa:K',
 'total_cloud_cover:p',
 'visibility:m',
 'wind_speed_10m:ms',
 'wind_speed_u_10m:ms',
 'wind_speed_v_10m:ms',]
X_train = pd.concat([clean_df(X_observed_a, selected_features), clean_df(X_estimated_a, selected_features)])
X_train = date_forcast_to_time(X_train)
X_train['almost_midnight'] = X_train['hour'].apply(assign_value)
X_train['sun_elevation_floored:d'] = X_train['sun_elevation:d'].apply(apply_floor_sun_elevation)
X_train['cloudy_day'] = ((X_train['effective_cloud_cover:p'] > 90) & (X_train['day_and_shadow'] == 1)).astype(int)
X_train.drop(columns = ['sun_elevation:d', 'effective_cloud_cover:p', 'day_and_shadow', 'is_day:idx', 'is_in_shadow:idx', 'hour'])
X_train, y_train = resize_training_data(X_train,y_a)

In [10]:
X_train.columns

Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'clear_sky_energy_1h:J', 'clear_sky_rad:W', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'is_day:idx', 'is_in_shadow:idx',
       'prob_rime:p', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'month', 'hour', 'hour_cos', 'day_and_shadow', 'almost_midnight',
       'sun_elevation_floored:d', 'cloudy_day'],
      dtype='object')

In [11]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Create the scorer
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Create the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=5)

# Fit the grid search
grid_search.fit(X_train, y_train["pv_measurement"])

# Get the best parameters
best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")

/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/simenfritzner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623

KeyboardInterrupt: 